# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store Quantum Circuit, experiment, build experiment, and excute them by local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy by local simulator.


## 1.1 Environment Setup

In [1]:
from qurry import EntropyMeasure, backendWrapper
from qiskit import (
    IBMQ, execute, transpile,
    QuantumRegister, ClassicalRegister, QuantumCircuit,
)

- Before we start, we need to import the package of Qurry and other packages we need. 

- In Qurry, we provider a simple way to import backend called `backendWrapper`, it will help us to import the backend we need, if your environment includes the GPU acceleration, `qiskit-aer-gpu`, it will be check the availability to access, if does, the GPU backend will also be available.

- The following chapter, we will told you how to use `backendWrapper` to import the real backend from your IBM account by `IBMProvider` or `AccountProvider`, and the more powerful class `backendManager` to import the real backend more easily.

- In this example, we will use normal `aer_simulator` as our backend


In [2]:
backend = backendWrapper()
backend('aer')

AerSimulator('aer_simulator_gpu')

In [3]:
print("| Does we have GPU backend to access:", backend.isAerGPU)
try:
    # If you have GPU, you can use this backend.
    print(backend('aer_gpu'))
except:
    pass

| Does we have GPU backend to access: True
aer_simulator_gpu


- Now, initialize our `EntropyMeasure`, we have two methods, `randomized` for Randomized Measure as default, and `hadamard` for Hadamard Test, these are our methods for measurement.

- In following tutorial, we will use `randomized` as our measurement method.

In [4]:
expProgram01 = EntropyMeasure(method='hadamard')
print(f'| The first method is "{expProgram01.__name__} which is `Hadamard Test`".')
expProgram02 = EntropyMeasure()
print(f'| The default method is "{expProgram02.__name__}" which is `Randomized Measure` we will use.')

| The first method is "qurrentHadamard which is `Hadamard Test`".
| The default method is "qurrentRandomized" which is `Randomized Measure` we will use.


---

## 1.2 Add Circuits to Qurry Object

- We use `trivialParamagnet` as our target to measure, and add it to our `Qurry` object, and import from `qurry.case` which there are some simple cases.

In [5]:
from qurry.case import trivialParamagnet, cat

In [6]:
sample01 = trivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01.circuit)
sample02 = cat(8)
print("| cat in 8 qubits:")
print(sample02.circuit)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘
| cat in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                        

In [8]:
expProgram02.add(sample01.circuit, 'trivialParamagnet')
expProgram02.add(sample02.circuit, 'cat')
expProgram02.add(sample02.circuit)
expProgram02.waves

<WaveContainer={
    trivialParamagnet: ...
    cat: ...
    2: ...
} with 3 waves load, a customized dictionary>

If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.

## 1.3 Excute Circuits

In [13]:
exp1 = expProgram02.measure(
    wave='trivialParamagnet',
    times=100,
    shots=1024,
)
exp1 

'305ed968-1464-4819-9fc6-5de3410da416'

In [14]:
expProgram02.exps

<ExperimentContainer={
    9d9d872e-bb46-4398-a2f4-3c4dca87bc0b: ...
    d9e48611-b99b-4afd-9417-12f516077b40: ...
    305ed968-1464-4819-9fc6-5de3410da416: ...
} with 3 experiments load, a customized dictionary>

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.

---

## 1.4 Result

And we want to know the Renyi Entropy of half system. Then

In [15]:
expProgram02.exps[exp1].analyze(
    (0, 4)
)

| Partition: (0, 4), Measure: (0, 8), 14 workers, 100 overlaps with mitigation. - 00:00 < 00:00


<qurrentRandomized.Analysis with serial=0, analysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), analysisContent(purity=0.9785594940185547, entropy=0.031268529046730005, and others), 0 unused arguments>

- You will found a report in `reports`

In [16]:
expProgram02.exps[exp1].reports

{0: <qurrentRandomized.Analysis with serial=0, analysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), analysisContent(purity=0.9785594940185547, entropy=0.031268529046730005, and others), 0 unused arguments>}

In [22]:
main, sideProdct = expProgram02.exps[exp1].reports[0].export()

- And there is it

In [23]:
main

{'purity': 0.9785594940185547,
 'entropy': 0.031268529046730005,
 'puritySD': 0.9659665600988488,
 'entropySD': 1.4241292169127489,
 'bitStringRange': [0, 4],
 'purityAllSys': 1.0142014884948731,
 'entropyAllSys': -0.020344296893044643,
 'puritySDAllSys': 1.0984090908248563,
 'entropySDAllSys': 1.562479809167018,
 'bitsStringRangeAllSys': [0, 8],
 'errorRate': -0.007103361431495614,
 'mitigatedPurity': 0.9656826557744141,
 'mitigatedEntropy': 0.0503789287650794,
 'num_qubits': 8,
 'measure': 'measure range: (0, 8)',
 'measureActually': [0, 8],
 'measureActuallyAllSys': [0, 8],
 'countsNum': 100,
 'input': {'degree': [0, 4], 'shots': 1024, 'unitary_loc': [0, 8]},
 'header': {'serial': 0,
  'datetime': '2023-05-31 23:17:13',
  'summoner': None,
  'log': {}}}

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.

In [24]:
sideProdct

{'purityCells': {0: 0.27643585205078125,
  1: 0.34393882751464844,
  2: 0.2898597717285156,
  3: 0.4466094970703125,
  4: 0.3080272674560547,
  5: 1.1817150115966797,
  6: 0.16468429565429688,
  7: 2.6594161987304688,
  8: 1.2433929443359375,
  9: 0.3359050750732422,
  10: 0.5522270202636719,
  11: 2.9485111236572266,
  12: 4.244045257568359,
  13: 0.11956024169921875,
  14: 0.28717041015625,
  15: 0.4899024963378906,
  16: 2.734048843383789,
  17: 3.8496475219726562,
  18: 0.16120529174804688,
  19: 0.6257266998291016,
  20: 0.20424652099609375,
  21: 0.16297340393066406,
  22: 0.5293331146240234,
  23: 0.5307464599609375,
  24: 0.6971721649169922,
  25: 2.411205291748047,
  26: 0.9134654998779297,
  27: 0.6405467987060547,
  28: 0.6970062255859375,
  29: 0.33347320556640625,
  30: 0.5040874481201172,
  31: 0.2035541534423828,
  32: 0.8049354553222656,
  33: 1.0982017517089844,
  34: 1.1844558715820312,
  35: 0.8387126922607422,
  36: 1.1491279602050781,
  37: 0.23339462280273438,
  3